In [ ]:
# default_exp retrieve

In [ ]:
import config
import requests
import base64
import json
import pandas as pd
import boto3

In [ ]:
#hide
from nbdev.showdoc import *

# retrieval

> Pull all tracks and their features for a user's public playlists

In [ ]:
# export
def cred():
    secret_name = "spotify_35"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    client_id = json.loads(get_secret_value_response['SecretString'])['spot_clientID']
    client_secret = json.loads(get_secret_value_response['SecretString'])['spot_clientSECRET']
    access_token = json.loads(get_secret_value_response['SecretString'])['spot_ACC']
    refresh_token = json.loads(get_secret_value_response['SecretString'])['spot_REF']

    return client_id, client_secret, access_token, refresh_token

In [ ]:
c_id, c_secret, access_token, refresh_token = cred()

In [ ]:
df_items = pd.DataFrame()

user_id = 'atraylor6'
offset = 0

while len(df_items) % 50 == 0:
    
    # print(len(df_items))

    play_url = f'https://api.spotify.com/v1/users/{user_id}/playlists?limit=50&offset={offset}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    r_play = requests.get(play_url, headers=headers)
    

    if r_play.status_code == 401:

        TOKEN_URL = 'https://accounts.spotify.com/api/token'

        message = config.CLIENT_ID + ':' + config.CLIENT_SECRET
        messageBytes = message.encode('ascii')
        base64Bytes = base64.b64encode(messageBytes)
        base64Message = base64Bytes.decode('ascii')
    
        headers = {
        'Authorization': 'Basic ' + base64Message,
        'Content-Type': 'application/x-www-form-urlencoded'
        }

        pars_refresh = {
            'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'redirect_uri': 'http://localhost:8888/callback',
        }

        r_refresh = requests.post(TOKEN_URL, headers=headers, params=pars_refresh)
        access_token = r_refresh.json()['access_token']

        headers = {
        'Authorization': f'Bearer {access_token}'
                    }

        r_play = requests.get(play_url, headers=headers)
        
    r_play = r_play.json()
    # print(r_play['items'][0])
    df_items = df_items.append(r_play['items'], ignore_index=True)
    # print(df_items.head())
    offset += 50



In [ ]:
df_pub = df_items[df_items['public'] == True]

In [ ]:
df_pub['id']

33    2iDp3O7zAdEdeUzlZHTtFu
35    0ZSKsOlj7QFX9LstS7fk39
39    2b6mA2TxfNjdnKH7mvgZIh
41    5zw1m0XWqZ1772z8tizUaD
42    4U9dVZ97mizvT2FBjx3fGx
43    29U1jhpnjkwziC4pcsvpml
44    4KgXBrWZqdqVMPrpFNHxpR
93    1gM83cehv98l8cdMnZmk5N
Name: id, dtype: object

In [ ]:
df_list = []

for p_id in df_pub['id']:

    df_tracks = pd.DataFrame()

    offset = 0

    while len(df_tracks) % 100 == 0:
        
        # print(len(df_tracks))

        track_url = f'https://api.spotify.com/v1/playlists/{p_id}/tracks?limit=100&offset={offset}'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        r_track = requests.get(track_url, headers=headers)

        track_ids = [t['track']['id'] for t in r_track.json()['items']]
        track_names = [t['track']['name'] for t in r_track.json()['items']]
        track_artists = [t['track']['artists'][0]['name'] for t in r_track.json()['items']]

        df_t = pd.DataFrame({
            'id': track_ids,
            'name': track_names,
            'artist': track_artists,
            'playlist id': p_id

        })  

        join_ids = ','.join(track_ids)
        feat_url = f'https://api.spotify.com/v1/audio-features?limit=100&offset={offset}&ids={join_ids}'
        r_feat = requests.get(feat_url, headers=headers)
        feat_frame = pd.DataFrame(r_feat.json()['audio_features'])

        df_t = pd.merge(df_t, feat_frame, on='id')

        df_tracks = df_tracks.append(df_t)
        offset += 100

    df_list.append(df_tracks)

df_tracks = pd.concat(df_list)

In [ ]:
id_map = df_pub[['id', 'name']].set_index('id').to_dict()['name']
df_tracks['playlist id'] = df_tracks['playlist id'].map(id_map)

In [ ]:
df_tracks.shape

(1147, 21)

In [ ]:
df_tracks.to_csv('track_features.csv')